# Importing request

In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
from config import api_key
from config import api_id
from config import api_key2

# API Call - Pollution Part 1
using a for loop to loop through three years work of PM2.5 data

In [2]:
state_code = '06'
county = '075'

response_sample = []
start = ["20190101", "20180101","20170101"]
end = ["20191231","20181231","20171231"]


for index in range(len(start)):
    # for each_state_code in ga_county_codes:
    url = f"https://aqs.epa.gov/data/api/sampleData/byCounty?email={api_id}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&state={state_code}&county={county}"
    response_sample.append(requests.get(url).json())

# Pulling Data and putting in list

In [3]:
time = []
date = []
county = []
lat = []
lon = []
site = []
sample = []


for x in range(0,3):
    for y in range(0,8688):
        time.append(response_sample[x]['Data'][y]['time_local'])
        date.append(response_sample[x]['Data'][y]['date_local'])
        county.append(response_sample[x]['Data'][y]['county'])
        lat.append(response_sample[x]['Data'][y]['latitude'])
        lon.append(response_sample[x]['Data'][y]['longitude'])
        site.append(response_sample[x]['Data'][y]['site_number'])
        sample.append(response_sample[x]['Data'][y]['sample_measurement'])

        
columns = ['time','date','county','lat','lon','site','sample']
df_sample = pd.DataFrame(data = list(zip(time,date,county,lat,lon,site,sample)), columns = columns)
df_sample.head()

,time,date,county,lat,lon,site,sample
0,23:00,2019-12-31,San Francisco,37.765946,-122.399044,0005,13.0
1,22:00,2019-12-31,San Francisco,37.765946,-122.399044,0005,15.0
2,21:00,2019-12-31,San Francisco,37.765946,-122.399044,0005,10.0
3,20:00,2019-12-31,San Francisco,37.765946,-122.399044,0005,8.0
4,19:00,2019-12-31,San Francisco,37.765946,-122.399044,0005,20.0


checking number of distinct sites

In [4]:
df_sample.site.unique()

array(['0005'], dtype=object)

# Grouping Hourly Data by day & Merging
PM2.5 data is organized by hour, but we need it on a daily level so it can map in with the AQI levels. First we do a groupby to get daily levels, then we need to remerge with the original data to get the categorical data back in (county, site, lat, lon)

In [5]:
df_sample1 = df_sample[['date','sample']].groupby('date').mean().reset_index().sort_values('date',ascending = False)
df_sample1.head()

,date,sample
1089,2019-12-31,9.909091
1088,2019-12-30,7.416667
1087,2019-12-29,6.416667
1086,2019-12-28,9.208333
1085,2019-12-27,8.208333


In [6]:
# data is all coming from same lat/lon and county
df_sample2 = df_sample.drop(columns = ['time','sample','site']).drop_duplicates().sort_values('date',ascending = False)
df_sample2.head()

,date,county,lat,lon
0,2019-12-31,San Francisco,37.765946,-122.399044
24,2019-12-30,San Francisco,37.765946,-122.399044
48,2019-12-29,San Francisco,37.765946,-122.399044
72,2019-12-28,San Francisco,37.765946,-122.399044
96,2019-12-27,San Francisco,37.765946,-122.399044


In [7]:
df_sample3 = pd.merge(df_sample2, df_sample1,
                     how = 'right',
                     on = 'date') 
df_sample3.sort_values('date',ascending = False).head()

,date,county,lat,lon,sample
0,2019-12-31,San Francisco,37.765946,-122.399044,9.909091
1,2019-12-30,San Francisco,37.765946,-122.399044,7.416667
2,2019-12-29,San Francisco,37.765946,-122.399044,6.416667
3,2019-12-28,San Francisco,37.765946,-122.399044,9.208333
4,2019-12-27,San Francisco,37.765946,-122.399044,8.208333


# Same process for PM2.5 now for AQI
AQI = Air Quality Index
This is not done every day but every 1 - 3 days depending on how the EPA decided to track it.

In [8]:
email = 'candice.sessa10@gmail.com'
api_key = 'dunfox31'
state_code = '06'
county = '075'

response_daily = []
start = ["20190101", "20180101","20170101"]
end = ["20191231","20181231","20171231"]


for index in range(len(start)):
    # for each_state_code in ga_county_codes:
    url = f"https://aqs.epa.gov/data/api/dailyData/byCounty?email={email}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&state={state_code}&county={county}"
    response_daily.append(requests.get(url).json())

In [9]:
response_daily

[{'Header': [{'status': 'Success',
    'request_time': '2020-05-14T21:15:39-04:00',
    'url': 'https://aqs.epa.gov/data/api/dailyData/byCounty?email=candice.sessa10@gmail.com&key=dunfox31&param=88101&bdate=20190101&edate=20191231&state=06&county=075',
    'rows': 1798}],
  'Data': [{'state_code': '06',
    'county_code': '075',
    'site_number': '0005',
    'parameter_code': '88101',
    'poc': 3,
    'latitude': 37.765946,
    'longitude': -122.399044,
    'datum': 'WGS84',
    'parameter': 'PM2.5 - Local Conditions',
    'sample_duration': '1 HOUR',
    'pollutant_standard': None,
    'date_local': '2019-01-01',
    'units_of_measure': 'Micrograms/cubic meter (LC)',
    'event_type': 'None',
    'observation_count': 24,
    'observation_percent': 100.0,
    'validity_indicator': 'Y',
    'arithmetic_mean': 3.875,
    'first_max_value': 14.0,
    'first_max_hour': 23,
    'aqi': None,
    'method_code': '170',
    'method': 'Met One BAM-1020 Mass Monitor w/VSCC - Beta Attenuation',


In [10]:
date = []
county = []
lat = []
lon = []
site = []
aqi = []


for x in range(0,3):
    for y in range(0,1772):
        date.append(response_daily[x]['Data'][y]['date_local'])
        aqi.append(response_daily[x]['Data'][y]['aqi'])

        
columns = ['date','aqi']
df_daily = pd.DataFrame(data = list(zip(date,aqi)), columns = columns).drop_duplicates()
df_daily.head()

,date,aqi
0,2019-01-01,NaN
1,2019-01-01,16.0
3,2019-01-02,NaN
4,2019-01-02,48.0
6,2019-01-03,NaN


In [11]:
# removing AQI NANs from dataset
df_daily1 = df_daily.dropna()
df_daily1.head()

,date,aqi
1,2019-01-01,16.0
4,2019-01-02,48.0
7,2019-01-03,71.0
16,2019-01-04,79.0
21,2019-01-05,25.0


# Merging AQI data with PM2.5 data

In [12]:
final = pd.merge(df_sample3, df_daily1,
              how = 'inner',
              on = 'date')
final.head()

,date,county,lat,lon,sample,aqi
0,2019-12-26,San Francisco,37.765946,-122.399044,7.043478,29.0
1,2019-12-25,San Francisco,37.765946,-122.399044,3.500000,15.0
2,2019-12-24,San Francisco,37.765946,-122.399044,5.333333,22.0
3,2019-12-23,San Francisco,37.765946,-122.399044,5.416667,23.0
4,2019-12-22,San Francisco,37.765946,-122.399044,4.666667,19.0


# Exporting Data to CSV

In [13]:
final.to_csv('aqi_pm25.csv')

# API Call Census Data
pulling on 5 polluted state/counties from random selection of the top 20 highest polluted counties in the US

pulling sectors for mining/quarring, utilities, construction, manufactoring, & wholesale trade

https://classcodes.com/naics-2-digit-sector-codes/

In [15]:
state = '06'
county = '075'
year = '2016'
variables_interest = ['NAICS2012_TTL,EMP,ESTAB']

sectors = ["31-33","21","22","42","48-49"]
base_url = f"https://api.census.gov/data/{year}/cbp?get={variables_interest[0]}&NAICS2012={sectors[0]}&NAICS2012={sectors[1]}&NAICS2012={sectors[2]}&NAICS2012={sectors[3]}&NAICS2012={sectors[4]}&for=county:{county}&in=state:{state}&key={api_key2}"
response = requests.get(base_url).json()

# Creating Dataset from Pulled Data
dropping first index as it is the column heading

In [16]:
columns = ['NAICS2012_TTL','EMP','ESTAB','NAICS2012',"State","County"]
df = pd.DataFrame(data = response,columns=columns)
df.drop(df.index[0])

,NAICS2012_TTL,EMP,ESTAB,NAICS2012,State,County
1,"Mining, quarrying, and oil and gas extraction",49,6,21,06,075
2,Utilities,0,15,22,06,075
3,Manufacturing,7894,725,31-33,06,075
4,Wholesale trade,15483,1183,42,06,075
5,Transportation and warehousing,8859,395,48-49,06,075
